## Hyper parameter tuning of embedding model

In [8]:
# # Uncomment this cell if running in colab
# !git clone github.com/leonleo997/virality-in-headliness
# !pip install virality-in-headliness/headlines

In [9]:
# If running directly from this notebook
!pip install ../

Processing /home/sebastian/Data_Science_Projects/virality-in-headliness
  Created wheel for headlines-project: filename=headlines_project-0.0.1-py3-none-any.whl size=12210 sha256=20f180074e6f935f99209e674642c6c855e3d6af96b737277408f886c8117ff3
  Stored in directory: /home/sebastian/.cache/pip/wheels/e2/24/0b/340bc509f1ec151908912726a4a40570bb7dd853685bf12092
Successfully built headlines-project
  Attempting uninstall: headlines-project
    Found existing installation: headlines-project 0.0.1
    Uninstalling headlines-project-0.0.1:
      Successfully uninstalled headlines-project-0.0.1


In [10]:
from headlines_project.models import TransformerEncoder
from headlines_project.data import create_data_pipeline
from headlines_project.utils import plot_attention_weights